## **Praktikum 2**

**Nasyawa Ramadhia // 2141720011 // 22**

Dataset yang digunkan adalah dataset Shakespeare's writing from Andrej Karpathy's The Unreasonable Effectiveness of Recurrent Neural Networks.

**Setup**
Import TensorFlow

In [1]:
import tensorflow as tf
import numpy as np
import os
import time

**penjelasan**:
- TensorFlow, library yang sering digunakan dalam pengembangan machine learning dan deep learning
- NumPy, library fundamental untuk komputasi numerik dalam Python, menyediakan struktur data (seperti array dan matriks)
- Modul os menyediakan fungsionalitas untuk berinteraksi dengan sistem operasi.
- Modul time menyediakan fungsi-fungsi terkait waktu dan pengukuran waktu dalam Python.


In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt','https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


penggunaan fungsi get_file dari modul tf.keras.utils dalam TensorFlow.
Fungsi ini digunakan untuk mengunduh file dari URL yang diberikan dan menyimpannya di dalam direktori yang ditentukan secara otomatis atau sesuai dengan kebutuhan.

**Load Data**

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


**penjelasan:**
- open(path_to_file, 'rb').read(),  fungsi open() untuk membuka file yang telah diunduh dalam mode 'rb' (mode baca untuk file dalam format biner). Kemudian menggunakan .read() untuk membaca seluruh isi file dalam bentuk biner.

- print(f'Length of text: {len(text)} characters') digunakan untuk mencetak panjang teks yang telah dibaca dari file

In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


**Olah Teks**

tf.keras.layers.StringLookup dapat mengubah setiap karakter menjadi ID numerik. Caranya adalah teks akan dipecah menjadi token terlebih dahulu.


In [6]:
example_texts = ['abcdefg', 'xyz']
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

membuat tf.keras,layers.StringLookup

mengconvert token menjadi id

In [7]:
ids_from_chars = tf.keras.layers.StringLookup(
vocabulary=list(vocab), mask_token=None)

**penjelasan**
- ids_from_chars adalah variabel yang menampung lapisan StringLookup.
- tf.keras.layers.StringLookup digunakan untuk membuat lapisan ini.
- parameter vocabulary yang harus berisi daftar karakter yang akan diindeks. vocab
- mask_token=None menunjukkan bahwa tidak ada token khusus yang ditetapkan sebagai mask

In [8]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [9]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

**penjelasan**
- chars_from_ids adalah variabel yang menampung lapisan StringLookup.
- invert=True menandakan bahwa konversi sekarang akan dilakukan dari ID kembali ke karakter aslinya.

Lapisan chars_from_ids ini berguna untuk mengonversi kembali ID yang dihasilkan sebelumnya ke karakter aslinya.

In [10]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [11]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [12]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

**penjelasan**:
Fungsi ini mengambil urutan ID (mungkin hasil dari suatu proses pada model atau proses pemrosesan sebelumnya) dan kemudian menggunakan chars_from_ids untuk mengonversi ID tersebut kembali menjadi karakter. Fungsi tf.strings.reduce_join digunakan untuk menggabungkan karakter-karakter ini menjadi sebuah string dalam bentuk teks.

**Membuat Trianing Set dan Target**

In [13]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [14]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [15]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [16]:
seq_length = 100

Metode batch memungkinkan Anda dengan mudah mengonversi karakter individual ini menjadi urutan ukuran yang diinginkan

In [17]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [18]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [19]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [20]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [21]:
dataset = sequences.map(split_input_target)

In [22]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


**Membuat Batch Training**


In [23]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

**Buat Model**

In [24]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

- tf.keras.layers.Embedding: Lapisan masukan. Tabel pencarian yang dapat dilatih yang akan memetakan setiap karakter-ID ke vektor dengan dimensi embedding_dim;
- tf.keras.layers.GRU: lapisan RNN dengan ukuran unit=rnn_units (Anda juga dapat menggunakan lapisan LSTM di sini.)
- tf.keras.layers.Dense: Lapisan keluaran, dengan keluaran vocab_size. Ini menghasilkan satu logit untuk setiap karakter dalam kosakata. Ini adalah log kemungkinan setiap karakter menurut model.

In [25]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [26]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [27]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

**Uji Model**

In [28]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [29]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [30]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [31]:
sampled_indices

array([41,  9, 35, 14, 39, 51, 33,  9, 51, 26, 19, 34, 53, 54, 42, 10, 13,
       61, 51, 34, 41, 21, 34, 38,  0, 23, 36, 49, 11, 25, 12, 14,  3, 24,
       15, 19, 43, 32, 32, 13, 62, 12, 41, 24, 42,  2, 20,  7, 51, 10, 20,
       48, 46, 35, 61, 41, 24, 54, 22, 22, 63,  5, 38, 61,  6, 18, 31, 47,
        0, 33, 64, 54, 45, 39,  8, 12,  5, 37, 11,  4, 17,  4,  5, 40, 45,
       49, 59,  4,  5, 24,  6, 15, 33, 65, 20, 56, 27, 41, 15, 40])

In [32]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'oy; I am past moe children, but thy sons and\ndaughters will be all gentlemen born.\n\nClown:\nYou are w'

Next Char Predictions:
 b"b.VAZlT.lMFUnoc3?vlUbHUY[UNK]JWj:L;A!KBFdSS?w;bKc G,l3GigVvbKoIIx&Yv'ERh[UNK]TyofZ-;&X:$D$&afjt$&K'BTzGqNbBa"


**Train Model**

In [33]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [34]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.189494, shape=(), dtype=float32)


In [35]:
tf.exp(example_batch_mean_loss).numpy()

65.9894

In [36]:
model.compile(optimizer='adam', loss=loss)

In [37]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

**Proses Training**

In [38]:
EPOCHS=20

In [39]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 14s 56ms/step - loss: 2.7117
Epoch 2/20
172/172 [==============================] - 11s 54ms/step - loss: 1.9816
Epoch 3/20
172/172 [==============================] - 11s 56ms/step - loss: 1.7024
Epoch 4/20
172/172 [==============================] - 12s 58ms/step - loss: 1.5441
Epoch 5/20
172/172 [==============================] - 12s 57ms/step - loss: 1.4486
Epoch 6/20
172/172 [==============================] - 12s 58ms/step - loss: 1.3812
Epoch 7/20
172/172 [==============================] - 12s 55ms/step - loss: 1.3289
Epoch 8/20
172/172 [==============================] - 12s 57ms/step - loss: 1.2835
Epoch 9/20
172/172 [==============================] - 11s 57ms/step - loss: 1.2431
Epoch 10/20
172/172 [==============================] - 11s 57ms/step - loss: 1.2031
Epoch 11/20
172/172 [==============================] - 11s 56ms/step - loss: 1.1628
Epoch 12/20
172/172 [==============================] - 11s 57ms/step - loss: 1.1202
E

**Generate Teks**

In [40]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

**penjelasan:**
Inisialisasi Kelas:
- Kelas OneStep menerima beberapa parameter seperti model, chars_from_ids, ids_from_chars, dan temperature
- model, yang akan digunakan untuk melakukan prediksi langkah demi langkah.
- temperature: Parameter opsional untuk mengatur "kreativitas" atau "uncertainty" dalam proses pengambilan sampel

Metode generate_one_step:
- Metode ini mengambil input sebagai teks (dalam bentuk string)
- teks input diubah menjadi token ID menggunakan ids_from_chars
- model dijalankan untuk mendapatkan predicted_logits, yang berisi kemungkinan probabilitas dari karakter-karakter selanjutnya.
- Dilakukan normalisasi menggunakan self.temperature untuk mengontrol variabilitas teks yang dihasilkan.
- Karakter selanjutnya diprediksi berdasarkan distribusi probabilitas menggunakan tf.random.categorical.

In [41]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [42]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
The collerts of the dishonoun after:
Is not the frowns of his aither persua, peace
With all when she deliver'd but thy nurse years device,
And once no less than Henry's heat? I hold it no harm.

FRIAR LAURENCE:
Bootless and two maid was not warrant;
But let my told it counsels muttle on.

ISABELLA:
To sin, for I have lively.

GREMIO:
I married, sir, no granted surpe, with ourself,
Whose honouration at your joys,
with words at good one.
Is not my father was the character, Here I stay,
For thinking thou did call him of his eyesigh stand:
Never was such filled for your pleasure is.
How long a sin which canst unthought doth embrace them
Maintain what you preat near all, as if a thousand fear
Thy love, black part: serve it; and I would discover him
To reap As all that I should be ingeg.

ADY GREY:
I shall, dischars; and tell them when I would do too,
Than with usurping breathing: here are like my father
Which of your hearts and heart-sorrow lightly.
The virgious confusion what the ca

In [43]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nAy, if no more than the town of France:\nHis loving trial o' the love have done,\nEven as has youth with hard but most have more cause\nTo more short unhappy words pretty into his life,\nOr death hath made them us, the ears illow;\nHer charity most good and so finely when too late!\n\nVOLUMNIA:\nBastain was that, my lord. I pity us,\nThe news that Rome gafe so? Come here; that's to read?\n\nPETRUCHIO:\nI shall.\n\nDUKE VINCENTIO:\nYou will not, then?\n\nPage:\nMy lord, here comes a neather of them: with trumpets' sun.\nEven such, the fruits of love I mean to see\nme from the sister of the king and no other right.\n\nROMEO:\nI take upon you; and, as it were ready!\nFor what often? have well indeed? how to Chrift again,\nBeing but off with the first dear father and\nHim and Saints between and nimble in life,\nA man of safframest morn beloved\nit to marry 'pardon'' is another one too, or not a suspect law,\nFrom when this time let fed this young blood speak again\nTo

**Ekspor Model Generator**

In [44]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

**penjelasan :**
- tf.saved_model.save(one_step_model, 'one_step'): Baris kode ini menyimpan model yang disebut one_step_model ke dalam file dengan nama 'one_step'
- one_step_reloaded = tf.saved_model.load('one_step'): Baris kode ini memuat (load) model yang sebelumnya telah disimpan dengan nama 'one_step'. Model yang dimuat kemudian diberikan label one_step_reloaded


In [45]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
Speak comest to London any of you have done.

Shepherd:
Lord, madam, go to, no, it doth so;
And I d


**penjelasan**

- Variabel states awalnya diatur ke None
- next_char diinisialisasi dengan teks awal "ROMEO:"
- Melalui loop for, dilakukan iterasi sebanyak 100 kali
- Setelah proses iterasi selesai, dilakukan penggabungan semua karakter dalam result menggunakan tf.strings.join() untuk membentuk teks berikutnya.

## **Tugas**

Prosedur pelatihan pada praktikum 2 merupakan prosedur sederhana, yang tidak memberi Anda banyak kendali. Model ini menggunakan "teacher-forcing" yang mencegah prediksi buruk diumpankan kembali ke model, sehingga model tidak pernah belajar untuk pulih dari kesalahan. Jadi, setelah Anda melihat cara menjalankan model secara manual, selanjutnya Anda akan mengimplementasikan custom loop pelatihan. Hal ini memberikan titik awal jika, misalnya, Anda ingin menerapkan pembelajaran kurikulum untuk membantu menstabilkan keluaran open-loop model. Bagian terpenting dari loop pelatihan khusus adalah fungsi langkah pelatihan.

Gunakan tf.GradientTape untuk men track nilai gradient. Anda dapat mempelajari lebih lanjut tentang pendekatan ini dengan membaca eager execution guide.

Prosedurnya adalah:
1. Jalankan Model dan hitung loss dengan tf.GradientTape.
2. Hitung update dan terapkan pada model dengan optimizer

In [48]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
    inputs, labels = inputs
    with tf.GradientTape() as tape:
      predictions = self(inputs, training=True)
      loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

Kode diatas menerapkan train_step method sesuai dengan  Keras' train_step conventions. Ini opsional, tetapi memungkinkan Anda mengubah perilaku langkah pelatihan dan tetap menggunakan keras Model.compile and Model.fit methods.

In [49]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [50]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [51]:
model.fit(dataset, epochs=1)

172/172 [==============================] - 14s 57ms/step - loss: 2.7316


Atau jika ingin lebih mengetahui dalamnya, kita bisa membuat custom training loop sendiri:

In [52]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

# saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

print()
print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.1973
Epoch 1 Batch 50 Loss 2.0948
Epoch 1 Batch 100 Loss 1.9349
Epoch 1 Batch 150 Loss 1.8574
Epoch 2 Batch 0 Loss 1.8245
Epoch 2 Batch 50 Loss 1.7728
Epoch 2 Batch 100 Loss 1.6935
Epoch 2 Batch 150 Loss 1.6981
Epoch 3 Batch 0 Loss 1.6297
Epoch 3 Batch 50 Loss 1.5459
Epoch 3 Batch 100 Loss 1.5686
Epoch 3 Batch 150 Loss 1.5253
Epoch 4 Batch 0 Loss 1.5276
Epoch 4 Batch 50 Loss 1.4719
Epoch 4 Batch 100 Loss 1.4908
Epoch 4 Batch 150 Loss 1.4224
Epoch 5 Batch 0 Loss 1.3849
Epoch 5 Batch 50 Loss 1.4119
Epoch 5 Batch 100 Loss 1.3711
Epoch 5 Batch 150 Loss 1.3855
Epoch 6 Batch 0 Loss 1.4148
Epoch 6 Batch 50 Loss 1.3013
Epoch 6 Batch 100 Loss 1.3171
Epoch 6 Batch 150 Loss 1.2855
Epoch 7 Batch 0 Loss 1.2576
Epoch 7 Batch 50 Loss 1.2844
Epoch 7 Batch 100 Loss 1.3062
Epoch 7 Batch 150 Loss 1.2736
Epoch 8 Batch 0 Loss 1.2212
Epoch 8 Batch 50 Loss 1.2445
Epoch 8 Batch 100 Loss 1.2574
Epoch 8 Batch 150 Loss 1.2446
Epoch 9 Batch 0 Loss 1.2130
Epoch 9 Batch 50 Loss 1.1671
Epoch 9

**PERBEDAAN ANTARA PRAKTIKUM 2 DAN TUGAS**

1. Perbedaan pertama adalah **waktu** karena pada praktikum 2 menggunakan
 **ephocs yang lebih banyak** sehingga waktu **runtime lebih lama** dibanding tugas

2. Pada praktikum **menggunakan function** yang telah disediakan oleh library tensorflow, sedangkan pada tugas **menggunakan perulangan dan tf**.GradientTape yang memungkinkan kita untuk lebih fleksibel pada penggunaan